# ArcGIS explorer
Find layers, list them by name + last updated date

In [64]:
import urllib, urllib.request
import json
import pandas as pd
from datetime import datetime, timedelta

def request(url, query=None, encoding=None):
    if not encoding:
        encoding = 'utf-8'
        
    url = urllib.parse.quote(url, safe=':/=?')    
    if query:
        url = "{}?{}".format(url, urllib.parse.urlencode(query))
    res = {}
    #req = urllib.request.Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(url) as f:
        res = f.read().decode(encoding)
        res = json.loads(res)
    return res

In [67]:
# data
url = 'https://services3.arcgis.com/66aUo8zsujfVXRIT/ArcGIS/rest/services'

In [74]:
print("URL = ", url)
# Get the list of services
services = request(url, {'f': 'pjson'})
collect_layers = []
for service in services.get('services', []):
    service_url = service.get('url')
    layers = request("{}/layers?f=pjson".format(service_url))
    for layer in layers.get('layers', []):
        collect_layers.append({            
            'name': layer.get('name'),
            'lastUpdate': layer.get('editingInfo', {}).get('lastEditDate'),
            'url': service_url + "/{}".format(layer.get('id'))
            'service': service.get('name')
        })
    for layer in layers.get('tables', []):
        collect_layers.append({            
            'name': layer.get('name'),
            'lastUpdate': layer.get('editingInfo', {}).get('lastEditDate'),
            'url': service_url + "/{}".format(layer.get('id')),
            'service': service.get('name') 
        })

df = pd.json_normalize(collect_layers)

URL =  https://services3.arcgis.com/66aUo8zsujfVXRIT/ArcGIS/rest/services


In [75]:
df['lastUpdate'] = pd.to_datetime(df['lastUpdate'],unit='ms')

url_df = df.loc[df['lastUpdate'] > (datetime.now() - timedelta(days=8))]
url_df = url_df.sort_values('lastUpdate', ascending=False)
# TODO: 
# [x] Filter out layers older than 1w
# [ ] Fetch column names for all the new ones
url_df.style.format({'url': lambda x: "<a href=\"{}\">link</a>".format(x)})

,name,lastUpdate,url,service
13,colorado_covid19_state_statistics_cumulative,2020-06-10 22:39:57.669000,link,colorado_covid19_state_statistics_cumulative
9,colorado_covid19_county_statistics_cumulative,2020-06-10 22:38:49.998000,link,colorado_covid19_county_statistics_cumulative
11,colorado_covid19_laboratory_positivity_data,2020-06-10 22:33:08.605000,link,colorado_covid19_laboratory_positivity_data
10,colorado_covid19_daily_state_statistics_cumulative,2020-06-10 22:29:09.649000,link,colorado_covid19_daily_state_statistics_cumulative
12,colorado_cdphe_covid19_county_id_rate,2020-06-10 22:00:38.115000,link,nan
7,active_cog315_jun2020,2020-06-09 19:25:17,link,nan
8,term_cog315_jun2020,2020-06-09 19:25:17,link,nan
17,courier_city_schedule,2020-06-08 15:31:50.460000,link,nan
2,Dental,2020-06-07 15:35:00.110000,link,nan
34,Streams_2020,2020-06-04 23:10:31.796000,link,nan


datetime.datetime(2020, 6, 2, 19, 48, 56, 320157)